In [1]:
import datetime as dt
import os

from config import SETTINGS

import seabeepy as sb

# Publish ODM mosaics from flight folders

 * Assumes the raw images have already been processed with NodeODM i.e. that `./orthophoto/odm_orthophoto.original.tif` already exists in the mission folder.
 
 * The folder/data structure must follow the guidelines [here](https://seabee-no.github.io/documentation/data-upload.html#sec-data-structure).

In [2]:
# List of mission folders to process. The trailing slash at the end of each path is required!
mission_fold_list = [
    r"/home/notebook/shared-seabee-ns9879k/seabirds/2023/Karmoy_Kavholmen_20230513/",
]

temp_dir = r"/home/notebook/cogs/"

In [3]:
def parse_mission_data(mission_name):
    """Extract region, area and date from folder name.

    Args
        mission_name: Str. Name of mission folder

    Returns
        Tuple or Bool. (region, area, date) if name can be parsed, else False.
    """
    try:
        region, area, date = mission_name.split("_")
    except ValueError:
        print(
            f"Skipping. Could not parse mission name '{mission_name}' (expected 'region_area_date')"
        )

        return False

    try:
        date = dt.datetime.strptime(date, "%Y%m%d")
    except ValueError:
        try:
            date = dt.datetime.strptime(date, "%Y%m%d%H%M")
        except ValueError:
            print(
                f"Skipping. Could not parse date '{date}' (expected 'YYYYMMDD' or 'YYYYMMDDhhmm')"
            )

            return False

    print(f"Data collected at {area} ({region}) on {date}.")

    return (region, area, date)

In [4]:
# Login to MinIO
minio_client = sb.storage.minio_login(
    user=SETTINGS.MINIO_ACCESS_ID, password=SETTINGS.MINIO_SECRET_KEY
)

In [5]:
for mission_fold in mission_fold_list:
    mission = os.path.basename(os.path.dirname(mission_fold))
    print("Processing:", mission)
    mission_data = parse_mission_data(mission)

    if mission_data:
        region, area, date = mission_data

        print("Preparing orthophoto for publishing.")
        ortho_path = os.path.join(
            mission_fold, "orthophoto", "odm_orthophoto.original.tif"
        )
        temp_path = os.path.join(temp_dir, mission + ".tif")
        sb.geo.standardise_orthophoto(
            ortho_path,
            temp_path,
            red_band=1,
            green_band=2,
            blue_band=3,
            nodata=255,
        )
        stan_path = os.path.join(mission_fold, "orthophoto", mission + ".tif")
        sb.storage.copy_file(temp_path, stan_path, minio_client, overwrite=False)
        os.remove(temp_path)

        print("Uploading to GeoServer.")
        layer_name = sb.geo.upload_raster_to_geoserver(
            stan_path,
            SETTINGS.GEOSERVER_USER,
            SETTINGS.GEOSERVER_PASSWORD,
            workspace="geonode",
        )

        print("Publishing to GeoNode.")
        sb.geo.publish_to_geonode(
            layer_name,
            SETTINGS.GEONODE_USER,
            SETTINGS.GEONODE_PASSWORD,
            workspace="geonode",
        )

        print("Updating metadata.")
        metadata = {
            "abstract": f"RGB mosaic collected by NINA at {area} ({region}) on {date}.",
            "date": date.isoformat(),
            "date_type": "creation",
            "attribution": "SeaBee",
        }
        sb.geo.update_geonode_metadata(
            layer_name,
            SETTINGS.GEONODE_USER,
            SETTINGS.GEONODE_PASSWORD,
            metadata,
        )

        print(f"Finished.\n")

Processing: Karmoy_Kavholmen_20230513
Data collected at Kavholmen (Karmoy) on 2023-05-13 00:00:00.
  Preparing orthophoto for publishing.
Input file size is 24445, 28860
0...10...20...30...40...50...60...70...80...90...100 - done.
  Uploading to GeoServer.
  Publishing to GeoNode.
  Updating metadata.
  Finished.
